In [ ]:
import math
import torch
import gpytorch
from sklearn.preprocessing import SplineTransformer, OneHotEncoder, PolynomialFeatures
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LassoCV
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
folder='/Users/user/PycharmProjects/PacManMain/data/Wt'

In [ ]:
# Define a class for Gaussian Process Model using GPyTorch
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
# Likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()

Load data

In [ ]:
psth=scipy.io.loadmat(folder+'/psths.mat')
X=scipy.io.loadmat(folder+'/X.mat')
drift=scipy.io.loadmat(folder+'/Y.mat')

Create design matrix + add in drift y

In [ ]:
#Get session psths and design mat
for sess in range(len(psth['psths'][0])-1):
    tmppsth=psth['psths'][0][sess]
    xtmp=X['Xflat'][0][sess]
    drifttmp=drift['driftflat'][0][sess]
    Xin=pd.DataFrame(np.concatenate((xtmp,drifttmp),axis=1))
    Xin.rename(columns={0:'wt',1:'reward',2:'time',3:'reldist',4:'relspeed',5:'drift'},inplace=True)
    
    #one hot encode reward level
    Xin_encoded = pd.get_dummies(Xin, columns=['reward'])
        
    # MinMax Scale continuous predictors
    scaler = MinMaxScaler()
    # List of columns to apply Min-Max scaling
    columns_to_scale = ['wt', 'time', 'reldist','relspeed']  # Replace with your column names
    
    # Apply the scaler only to the specified columns
    Xin_encoded[columns_to_scale] = scaler.fit_transform(Xin_encoded[columns_to_scale])
    Xtrain=torch.tensor(np.array(Xin_encoded))
    #Do GP regression to get tuning
    Ytrain=torch.tensor(tmppsth[:,0])
    model = ExactGPModel(Xtrain, Ytrain, likelihood)
    
    
        # Find optimal model hyperparameters
    model.train()
    likelihood.train()
    
    # Use the Adam optimizer
    optimizer = torch.optim.Adam([
        {'params': model.parameters()},  # Includes GaussianLikelihood parameters
    ], lr=0.1)
    
    # Loss function (the marginal log likelihood)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    
    training_iterations = 50
    for i in range(training_iterations):
        optimizer.zero_grad()
        # Output from model
        output = model(Xtrain)
        # Calc loss and backprop gradients
        loss = -mll(output, Ytrain)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()


            